In [138]:
class Stalk:
    def __init__(self,stalk):
        self.stalk = stalk
        if len(self.stalk) == 0: self.stalk = '$'

    def __repr__(self): return self.stalk

    def __eq__(self,other): return self[0] == str(other)[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.stalk[index]

    def __len__(self):
        if self.stalk == '$' or self.stalk == '^': return 0
        return len(self.stalk)
    
    def common_substring(self,other):
        i = 0
        substr = ''
        if type(other) == str: other = Stalk(other)
        while i < min(len(self),len(other)) and self[i] == other[i]:
            substr += self[i]
            i += 1
        return Stalk(substr), Stalk(self[i:]), Stalk(other[i:])

In [139]:
class Leaf:
    def __init__(self,left,right=''):
        if len(left) == 0:
            self.left = left
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(right)
    
    def __repr__(self): return str(self.right)

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self.left)

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)#+len(self.right)

In [140]:
class Branch:
    def __init__(self):
        self.b = {}
        self.s = {}

    def __repr__(self): return repr(self.b)

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index):
        if type(index) == str: return self.b[Stalk(index)]
        return self.b[index]
    
    def __is_shallow__(self):
        for a in self.b.values():
            if type(a) == Branch: return False
        return True
    
    def __traverse__(self,context):
        b = self[context[0]]
        s = self.s[context[0]]
        context = context[len(s[0]):]
        while len(context) > 0 and len(b) > 1:
            s = b.s[context[0]]
            b = b[context[0]]
            context = context[len(s[0]):]
        return b
    
    def __setitem__(self,index,value):
        if type(index) == str: self.b[Stalk(index)] = value
        else: self.b[index] = value

    def __contains__(self,other): 
        if type(other) == str: return Stalk(other) in self.b
        return other in self.b

    def __len__(self): return len(self.b)

    def pop(self,index): return self.b.pop(index)

    def pop_copy(self,index):
        b = self.b.copy()
        b.pop(index)
        s = self.s.copy()
        s.pop(index)
        br = Branch()
        br.b = b
        br.s = s
        return br

    def add(self,stalk,reads):
        # if stalk.stalk == 'etty_bought':
        #     print()
        if stalk in self:
            if not len(stalk):
                self[stalk].right+=1
                return
            if type(self[stalk]) == Leaf:
                branch = Branch()
                l1 = self.pop(stalk)
                stalk_ = list(self.s.pop(stalk))
                stalk_[0],l1.left,l2 = stalk_[0].common_substring(stalk)
                branch.add(l1.left,stalk_[1].copy())
                branch.add(l2,reads)
                stalk_[1].update(reads)
                stalk_ = tuple(stalk_)
                self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
            else:
                stalk_ = list(self.s.pop(stalk))
                branch = self.pop(stalk)
                stalk_[0],bstalk,stalk = stalk_[0].common_substring(stalk)
                if len(bstalk):
                    br = Branch()
                    br[bstalk] = branch 
                    br.s[bstalk] = (bstalk,stalk_[1].copy())
                    br.add(stalk,reads)
                    self[stalk_[0]] = br
                else: 
                    branch.add(stalk,reads)
                stalk_[1].update(reads)
                stalk_ = tuple(stalk_)
                if not len(bstalk): self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
        else:
            if type(stalk) == str: stalk = Stalk(stalk)
            self.s[stalk] = (stalk,reads)
            self[stalk] = Leaf(stalk)

    def unpack(self,exclude,context,t=None):
        if t is None: t = ''
        ambiguities = {}
        for s in self.s:
            for b in self[s].b.values():
                if type(b) == Branch: return ambiguities.update(b.unpack(exclude,context,t=s))
                if len(self[s].s[b.left][1].intersection(exclude)): continue
                ambiguities[list(self[s].s[b.left][1])[0]] = context, Sequence(t + s.stalk + b.left.stalk,{list(self[s].s[b.left][1])[0]})
                t = ''
        return ambiguities

In [164]:
class Sequitur:
    def __init__(self,reads):
        self.branch = Branch()
        self.reads = reads
        for read in reads: 
            for i in range(len(read)): self.branch.add(Stalk(read[i:]),{read})
        self.sequence = ''
    
    def resolve_ambiguities(self,sequence):
        if len(sequence.ambiguities) == 1:
            ext = list(sequence.ambiguities.values())[0]
            if sequence.seq.endswith(ext[0]): 
                sequence += ext[1]
                return
        # for a in sequence.ambiguities:

        pass

    def sequitur(self,sequence):
        self.sequence = sequence
        i = 1
        context = self.sequence.seq[-i:]
        while i <= len(self.sequence.seq) and len(set(self.reads).difference(self.sequence.seen)):
            branch = self.branch.__traverse__(context)
            if branch.__is_shallow__():
                stalks = branch.s.copy()
                stalks.pop('$')
                if len(stalks) == 1:
                    stalks = list(stalks.values())
                    stalks = stalks[0]
                    if len(self.sequence.seen.intersection(stalks[1])):
                        i -= 1
                        context = self.sequence.seq[-i:]
                        branch = self.branch.__traverse__(context).pop_copy('$')
                        self.sequence.ambiguities.update(branch.unpack(self.sequence.seen,context))
                        self.resolve_ambiguities(self.sequence)
                        i = 1
                        context = self.sequence.seq[-i:]
                        continue
                else:
                    for s in stalks.values():
                        for x in s[1].difference(self.sequence.seen.intersection(s[1])):
                            self.sequence.ambiguities.update({x:(context,Sequence(s[0].stalk,{x}))})
                    self.resolve_ambiguities(self.sequence)
                    i = 1
                    context = self.sequence.seq[-i:]
                    continue
                branch = branch.b.copy()
                branch.pop('$')
                if len(branch) > 1:
                    options += list(branch)
                    break
                branch = list(branch)[0]
                self.sequence.seq += branch.stalk
                self.sequence.seen.update(stalks[1])
                i = 0
            i += 1
            context = self.sequence.seq[-i:]

    def assemble(self,start=None):
        pass

In [142]:
class Sequence:
    def __init__(self,seq=None,seen=None):
        self.seen = set()
        self.ambiguities = {}
        if seq is None: self.seq = ''
        else: 
            self.seq = seq
            if seen is None: self.seen.add(seq)
            else: self.seen = seen

    def __add__(self,other):
        self.seq += other.seq
        self.seen.update(other.seen)
        for a in set(self.ambiguities).intersection(self.seen):
            self.ambiguities.pop(a)
        for a in set(other.ambiguities).intersection(self.seen):
            other.ambiguities.pop(a)
        self.ambiguities.update(other.ambiguities)
        return self

In [165]:
reads = ['betty_bought_butter_th','tter_the_butter_was_','he_butter_was_bitter_','as_bitter_betty_bought','tty_bought_better_butter_t','r_butter_to_make_the_','ke_the_bitter_butter_better']
sequitur = Sequitur(reads)

In [166]:
sequitur.sequitur(Sequence('betty_bought_butter_th'))

In [167]:
sequitur.sequence.seq

'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'